# Process GAM model results

In [1]:
%load_ext autoreload
%autoreload 2
%cd D:\mobi-social-segregation-se

D:\mobi-social-segregation-se


In [40]:
# Load libs
import pandas as pd
from lib import helpers

## 1. Go through the three models

In [46]:
perf_dict_list = []
for md, s in zip(('all', 'low_income', 'high_income'),
                 ('all', 'Low income', 'High income')):
    path2result = f'D:/mobi-social-segregation-se/results/ebm/'
    gam_results = helpers.EBMResultsOrganizer(file_loc=path2result + f'model_{md}.p')
    gam_results.load_raw_data(select=s)

    # Feature performance
    perf = gam_results.performance()
    perf['model'] = md
    perf_dict_list.append(perf)

    # Feature importance
    df_f = gam_results.feature_importance()
    df_f.to_csv(f'results/ebm/{md}/f_score.csv', index=False)

    # Load feature effects
    gam_results.feature_scores()
    df_fscore_f = gam_results.single_feature_effect()
    df_fscore_f.to_csv(f'results/ebm/{md}/features.csv', index=False)
    interaction_path = f'D:/mobi-social-segregation-se/results/ebm/{md}/interactions/'
    gam_results.interection_effect(path2save=interaction_path)

Label low, middle, and high income.
Label low, middle, and high income.
Label low, middle, and high income.


In [47]:
df_perf = pd.DataFrame(perf_dict_list)
df_perf

,rmse_train,r2_train,rmse_test,r2_test,model
0,0.064368,0.347802,0.064110,0.348418,all
1,0.072892,0.485120,0.072824,0.490872,low_income
2,0.063057,0.291117,0.062901,0.278699,high_income


## 2. Feature importance

In [75]:
df_list = []
for md, s in zip(('low_income', 'high_income'),
                 ('Low income', 'High income')):
    df = pd.read_csv(f'results/ebm/{md}/f_score.csv')
    df.loc[:, 'Income group'] = s
    df.loc[:, 'Rank'] = range(1, len(df) + 1)
    df_list.append(df)
df_f = pd.concat(df_list).sort_values(by=['Income group', 'Score', 'Name'], ascending=False)
features2keep = df_f.loc[df_f.Rank < 11, 'Name'].unique()
df_f

,Name,Color,Score,Income group,Rank
0,Prob. born outside Sweden,steelblue,0.013175,Low income,1
1,Median distance from home,coral,0.011661,Low income,2
2,Prob. in lowest income group,steelblue,0.011375,Low income,3
3,Residential income segregation,steelblue,0.008986,Low income,4
4,Job accessibility by car,darkgreen,0.008485,Low income,5
5,No. of unique locations visited,coral,0.007472,Low income,6
6,Transit accessibility by walking,darkgreen,0.007136,Low income,7
7,Prob. of owning a car,steelblue,0.007021,Low income,8
8,Job density at residence,darkgreen,0.006594,Low income,9
9,Average displacement,coral,0.006176,Low income,10


In [76]:
df_f2plot = pd.merge(df_f[df_f['Income group'] == 'Low income'].
                drop(columns=['Income group']).
                rename(columns={'Score': 'Low income'}),
                df_f[df_f['Income group'] == 'High income'].
                drop(columns=['Income group', 'Rank']).
                rename(columns={'Score': 'High income'}),
                on=['Name', 'Color'],
                how='outer').fillna(0)
df_f2plot = df_f2plot.loc[df_f2plot.Name.isin(features2keep)]
df_f2plot

,Name,Color,Low income,Rank,High income
0,Prob. born outside Sweden,steelblue,0.013175,1.0,0.002899
1,Median distance from home,coral,0.011661,2.0,0.003475
2,Prob. in lowest income group,steelblue,0.011375,3.0,0.005530
3,Residential income segregation,steelblue,0.008986,4.0,0.005492
4,Job accessibility by car,darkgreen,0.008485,5.0,0.005881
5,No. of unique locations visited,coral,0.007472,6.0,0.008585
6,Transit accessibility by walking,darkgreen,0.007136,7.0,0.003824
7,Prob. of owning a car,steelblue,0.007021,8.0,0.003514
8,Job density at residence,darkgreen,0.006594,9.0,0.006398
9,Average displacement,coral,0.006176,10.0,0.004350


In [77]:
df_f2plot.to_csv('results/ebm/feature_importance.csv', index=False)